#サンプリングによる制限付ボルツマンマシンの学習
QUBOで計算できる用途として組合せ最適化問題以外にも、最適解や局所解を含む解の確率分布モデルを利用したボルツマンマシンがあります。ここでは、このボルツマンマシンにおけるサンプリング方法を簡単なモデルで確認したいと思います。

##RBM（制限付ボルツマンマシン）とは？
RBMは制限ボルツマンマシン、制限付きボルツマンマシンなどと呼ばれ、ボルツマンマシンと呼ばれるタイプのNNに層を制限して2層の可視層と隠れ層に制限したものです。

参考：
Restricted Boltzmann machine
https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine

##参考資料
モントリオール大学のこの論文が始まりです。ディープラーニングでの有名な先生がD-Waveでの深層学習の実装可能性として下記の論文を発表し、D-Waveマシンへの実装が検討されました。

On the Challenges of Physical Implementations of RBMs
Vincent Dumoulin and Ian J. Goodfellow and Aaron Courville and Yoshua Bengio
https://arxiv.org/pdf/1312.5258.pdf

この際にはD-Waveを模したソフトウェアシミュレータでNLLとよばれる学習の際に必要なRBMの勾配の推定にサンプリングが活用できるという方針は模索されましたが、実機は使われませんでした。この論文とGeoffrey Hinton先生の下記の論文はRBMを学ぶ上で大変役立ちます。

A Practical Guide to Training Restricted Boltzmann Machines
Geoffrey Hinton
https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf

##ボルツマンサンプリングの提唱
次に、実際に上記の問題をD-Wave実機で行なった結果として、

Application of Quantum Annealing to Training of Deep Neural Networks
Steven H. Adachi, Maxwell P. Henderson
https://arxiv.org/abs/1510.06356

があります。これは量子アニーラをボルツマンサンプリングマシンとして、上記のNLLの推定に役立てようという具体的な方針が示され、実機で学習が実際にされています。

##RBMモデル概略
まず、モデルは可視層と呼ばれる層と隠れ層と呼ばれる２層からなるモデルです。結合に方向性がなく無向グラフです。

<img src="./img/021_0.png">

結合分布が下記のようにギブス分布、ボルツマン分布に従います。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=p(v%2Ch)%20%3D%20%5Cfrac%7B1%7D%7BZ%7Dexp(-E(v%2Ch))">

ここで、この結合分布はエネルギー関数で規定され、可視層のノード数nと隠れ層のノード数mとで下記のように示されます。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=E(v%2Ch)%20%3D%20-%5Csum_%7Bi%3D1%7D%5Enb_iv_i-%5Csum_%7Bj%3D1%7D%5Emc_jh_j-%5Csum_%7Bi%3D1%7D%5En%5Csum_%7Bj%3D1%7D%5EmW_%7Bij%7Dv_ih_j">

また、正規化のための規格化定数、分配関数は下記の通りになります。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=Z%20%3D%20%5Csum_%7Bv_k%7D%5Csum_%7Bh_l%7Dexp%5Cleft(%5Csum_kb_kv_k%20%2B%20%5Csum_lc_lh_l%20%2B%20%5Csum_%7Bk%2Cl%7DW_%7Bkl%7Dv_kh_l%20%20%5Cright)">

また、完全二部グラフより、条件付き確率分布はそれぞれ、vとhについてシグモイド関数を用いて下記の通りになります。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=P(h_j%3D1%7Cv)%20%3D%20sigm(c_j%2B%5Csum_iW_%7Bij%7Dv_i)">

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=P(v_i%3D1%7Ch)%20%3D%20sigm(b_i%2B%5Csum_jW_%7Bij%7Dh_j)">

##学習について
次に上記の確率分布からなるNNのモデルの学習方法を確認したいと思います。複層のDBN（DBM）でも、RBMの形にして学習をします。これらの学習はlogPを最大にするように、トレーニングデータとモデルの誤差計算をします。結合係数やバイアスの勾配計算はlogPを用いて下記のようになります。

<img src="./img/021_1.png">

ここで、上記の結合係数の勾配計算で、modelの期待値の計算はあまり効率的な計算がありません。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=%3D%20%5Cfrac%7B1%7D%7BZ%7D%5Csum_%7B%5C%7Bv_k%5C%7D%7D%5Csum_%7B%5C%7Bh_l%5C%7D%7Dv_ih_j%20exp%5Cleft(%5Csum_kb_kv_k%20%2B%20%5Csum_lc_lh_l%2B%5Csum_%7Bkl%7DW_%7Bkl%7Dv_kh_l%5Cright)%0D%0A">

実際には、この値を直接求めるのが大変なので、近似計算でCD法など、Gibbsサンプリングを応用した手法で、順番に隠れ層と可視層の計算を行なって値を取るという方法が行われます。CD法などでは計算量を節約する意味で、かなり近似的な計算が行われ、これを精度改善しようとすると計算量と時間がかかります。

<img src="./img/021_2.png">

上記は結合係数の更新ですが、バイアスの更新でも同様です。

##ボルツマンサンプリングを活用したパラメータ更新
ここで、このCD法の代わりにアニーリングマシンを使って元の勾配計算どおりしまおうというのが実機を使ったボルツマンサンプリング学習の基本です。

ボルツマンサンプリングを使った際の結合係数やバイアスの更新式は、下記の通りです。αはモーメンタムでϵは学習率です。

<img src="./img/021_3.png">

RBMを学習した後は古典計算機でバックプロパゲーションで仕上げの学習をするようです。

##アニーリングを活用したサンプリング手法

D-Waveマシンは量子アニーリングという理論をベースに作られており、基本的には最適化問題を解くために最小値問題を求めるような設計になっています。しかし実際には、外部環境の影響やその他の理由で、最適解に落ち着かない場合が多いです。そのため、この局所解へ落ちる性質を利用して、分布を求めるサンプリングマシンとして活用するというのが、サンプリング学習の基本です。

この励起状態のばらつきがボルツマン分布として仮定できると、下記式に近似できこれと初期に出てきた分布の式を対応させることで、サンプリング手法を更新式に導入できそうです。

<img src="./img/021_4.png">

Hfは最終的に求めるコスト関数で、βeffはサンプリングの分布を調整する変数です。これを使用することで、一番計算量のかかる部分を下記のように近似します。

<img src="./img/021_5.png">

これをmodel期待値に適用します。他の可視層や隠れ層のモデル期待値も同様です。



##簡単な例題で実行する
簡単な例題で上記のRBMのサンプリングに関して学んでみたいと思います。
まずはD-Waveの例題でも載っている、簡単な問題で。

<img src="./img/021_6.png">

このQUBOmatrixの場合、コスト関数は、

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=E(x)%20%3D%20-x_1-x_2%2B2x_1x_2">


となります。xの場合の数でエネルギーを求めて見ると、

<img src="./img/021_7.png">

解析的にかんがえて見ると、ここではすべての場合の数が出せているので、まず規格化定数Zを求めます。

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=Z%20%3D%20%5Csum%20exp(E(x))%20%3D%20exp(0)%20%2B%20exp(1)%20%2B%20exp(1)%20%2B%20exp(0)%20%3D%201%2B2.718%2B2.718%2B1%20%3D%207.44">

そして、

<img class="math math-inline" src="https://render.githubusercontent.com/render/math?math=%7BP%20%3D%20%5Cfrac%7B1%7D%7BZ%7Dexp(E)%0D%0A%7D">

より、確率は0.13と0.37がでます。

<img src="./img/021_8.png">

これを手作業で出すのは大変なので、サンプリングで考えてみます。

##Wildqatでサンプリング
サンプリングは簡単です。上記QUBOを複数回実行して分布をとります。

In [0]:
!pip install -U wildqat

In [0]:
import wildqat as wq
import numpy as np
a = wq.opt()

beta = 0.05
a.R = 0.5

a.qubo = np.asarray([[-1,2],[0,-1]])*beta
qarr = [[0,0],[0,1],[1,0],[1,1]]

In [25]:
cnt = [0]*4

for i in range(1000):
  b = a.sa()
  
  for j in range(4):
    if b == qarr[j]:
      cnt[j] += 1
      
print(cnt)

0.12228679656982422
0.10138988494873047
0.10092401504516602
0.11755251884460449
0.10254478454589844
0.10146760940551758
0.0953066349029541
0.0992879867553711
0.10161328315734863
0.09867191314697266
0.09425115585327148
0.09727263450622559
0.09310007095336914
0.09523272514343262
0.098907470703125
0.09640812873840332
0.09568166732788086
0.09816360473632812
0.09549927711486816
0.09382247924804688
0.10044288635253906
0.10060834884643555
0.09703946113586426
0.10304403305053711
0.10109734535217285
0.10042309761047363
0.11520218849182129
0.0980985164642334
0.10106158256530762
0.13313984870910645
0.09820842742919922
0.10322237014770508
0.11213541030883789
0.1018531322479248
0.14796686172485352
0.10197973251342773
0.10377860069274902
0.10607266426086426
0.10097956657409668
0.14933323860168457
0.09900331497192383
0.10243058204650879
0.10419464111328125
0.10205698013305664
0.1443641185760498
0.10746097564697266
0.11792755126953125
0.11324191093444824
0.09914398193359375
0.11715340614318848
0.10086

 上記はqarr = [[0,0],[0,1],[1,0],[1,1]]の出現確率を表しており、エネルギーの低い方が出やすくなっています。
 全体の試行回数は今回1000だったので、それぞれを1/1000することで確率分布を求めることができます。これを更新式に入れることで結合荷重の更新を行います。
 
 <img src="./img/021_9.png">